# Inference

## Dependencies and imports

In [ ]:
import os
from abc import ABC, abstractmethod

import pandas as pd
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

## Utility

In [ ]:
class AbstractModel(ABC):
    PROMPT = "Sei un dipendente pubblico che deve riscrivere dei documenti istituzionali italiani per renderli semplici e comprensibili per i cittadini. Ti verrà fornito un documento pubblico e il tuo compito sarà quello di riscriverlo applicando regole di semplificazione senza però modificare il significato del documento originale. Ad esempio potresti rendere le frasi più brevi, eliminare le perifrasi, esplicitare sempre il soggetto, utilizzare parole più semplici, trasformare i verbi passivi in verbi di forma attiva, spostare le frasi parentetiche alla fine del periodo."

    def __init__(self, hugging_face_model_id: str, torch_dtype=torch.bfloat16, quantization_config=None):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using {self.device} for inference")

        self.tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_id, token=os.getenv("HF_TOKEN"))
        self.model = AutoModelForCausalLM.from_pretrained(hugging_face_model_id,
                                                          trust_remote_code=True,
                                                          device_map=self.device,
                                                          torch_dtype=torch_dtype,
                                                          token=os.getenv("HF_TOKEN"),
                                                          quantization_config=quantization_config).eval()
        print("Model loaded")

    @abstractmethod
    def build_prompt(self, _text_to_simplify):
        pass

    @abstractmethod
    def decode(self, _decoded):
        pass

    def predict(self, _texts_to_simplify):
        prompts = [self.build_prompt(_text) for _text in _texts_to_simplify]
        outputs = []
        for prompt in tqdm(prompts):
            x = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)
            y = self.model.generate(x, max_new_tokens=512, temperature=0.2, top_p=0.1, do_sample=True, eos_token_id=self.tokenizer.eos_token_id, pad_token_id=self.tokenizer.eos_token_id)
            decoded = self.tokenizer.batch_decode(y, skip_special_tokens=False)
            decoded = [self.decode(d) for d in decoded]
            outputs.extend(decoded)
        return outputs

In [ ]:
class Llama3(AbstractModel):
    HUGGING_FACE_MODEL_ID = "DeepMount00/Llama-3-8b-Ita"

    def __init__(self):
        super().__init__(Llama3.HUGGING_FACE_MODEL_ID, torch.bfloat16)
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def build_prompt(self, _text_to_simplify):
        messages = [
            {"role": "system", "content": Llama3.PROMPT},
            {"role": "user", "content": _text_to_simplify},
        ]
        return self.tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

    def decode(self, _decoded):
        return _decoded.split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].split('<|eot_id|>')[0].strip()

## Load datasets

In [ ]:
df = pd.read_csv('./texts/original.csv', encoding='utf-8')

## Model

In [ ]:
model = Llama3()

## Random predictions

In [ ]:
for s in df.sample(1).to_dict(orient='records'):
  output = model.predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\nmodel: ", output)
  print('----------------')

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 1/1 [00:12<00:00, 12.63s/it]


original:  I procedimenti amministrativi oggetto dei Servizi di Polizia Locale Reparto Operativo sono indicati nel prospetto di seguito riportato. 

model:  I Servizi di Polizia Locale Reparto Operativo gestiscono i seguenti procedimenti amministrativi: 

- Verifica dei documenti

- Controllo delle attività

- Verifica delle informazioni

- Verifica delle attività

- Verifica delle informazioni.
----------------


## Run all predictions

In [ ]:
df['simplified_text'] = model.predict(df['original_text'].tolist())

100%|██████████| 619/619 [1:24:27<00:00,  8.19s/it]


## Save simplified datasets

In [ ]:
df.to_csv('./texts/llama3.csv', index=False)